<a href="https://colab.research.google.com/github/asmaakhaledd/PID-NN/blob/PID-Inference/Inference_of_Opt_PID_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install numpy pandas tensorflow

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
import xml.etree.ElementTree as ET
from tensorflow.keras.models import load_model
from datetime import datetime

Load model

In [23]:
def load_model(model_path):
    # Load the model without compiling it initially
    pid_model = tf.keras.models.load_model(model_path, compile=False)
    pid_model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())  # Recompile with the correct loss
    return pid_model

Parse XML data into DataFrame

In [29]:
def parse_xml(file_path):
    import xml.etree.ElementTree as ET

    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []

    # Extract patient weight
    weight = float(root.get('weight', 0))  # Defaults to 0 if missing

    # Extract meal events
    meal_events = []

    meal_node = root.find('meal')
    if meal_node:
        for event in meal_node.findall('event'):
            meal_events.append({
                'timestamp': datetime.strptime(event.get('ts'), "%d-%m-%Y %H:%M:%S"),
                'carbs': float(event.get('carbs', 0))
            })
    # Process glucose levels
    for event in root.find('glucose_level'):
        timestamp = datetime.strptime(event.get('ts'), "%d-%m-%Y %H:%M:%S")
        glucose = float(event.get('value'))

        # Find closest meal within 2 hours
        meal_intake = next(
            (m['carbs'] for m in reversed(meal_events) if (timestamp - m['timestamp']).total_seconds() <= 7200),
            0.0
        )

        data.append({
            'timestamp': timestamp,
            'glucose': glucose,
            'meal_carbs': meal_intake,
            'weight': weight
        })

    df = pd.DataFrame(data)
    return df.sort_values('timestamp')


Preprocess time features (cyclic encoding)

In [30]:
def preprocess_time_features(df):
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    df['time_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['time_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    return df.drop(['timestamp', 'hour', 'minute'], axis=1)

Prepare sequences for PID tuning model (input data for LSTM)

In [31]:
def prepare_pid_training_data(df, sequence_length=30):
    X_pid, y_pid = [], []
    for i in range(len(df) - sequence_length):
        glucose_error = df['glucose'].iloc[i] - 110
        glucose_change = df['glucose'].iloc[i] - df['glucose'].iloc[i-1] if i > 0 else 0

        Kp = 0.05 * np.log(1 + abs(glucose_error))
        Ki = 0.005 * np.log(1 + abs(glucose_error))
        Kd = 0.002 * np.log(1 + abs(glucose_change))

        X_pid.append([glucose_error, glucose_change, df['meal_carbs'].iloc[i], df['weight'].iloc[i],
                      df['time_sin'].iloc[i], df['time_cos'].iloc[i]])
        y_pid.append([Kp, Ki, Kd])

    return np.array(X_pid), np.array(y_pid)

Predict Insulin Dosage

In [34]:
def predict_insulin_dosage(model, test_df):
    # Prepare the data
    X_pid_test, _ = prepare_pid_training_data(test_df)

    # Reshape for LSTM
    X_pid_test_reshaped = X_pid_test.reshape((X_pid_test.shape[0], X_pid_test.shape[1], 1))

    # Predict PID gains
    pid_gains = model.predict(X_pid_test_reshaped)

    # Loop through samples and compute insulin dosages
    num_samples = len(X_pid_test)  # Use the length of X_pid_test, not test_df
    for i in range(num_samples):
        glucose_prediction = X_pid_test[i][0]  # Assuming glucose is the first feature in the sequence
        glucose_error = glucose_prediction - 110
        glucose_change = glucose_prediction - X_pid_test[i-1][0] if i > 0 else 0

        weight = test_df['weight'].iloc[i]  # Get weight from test dataframe
        ISF = 5000 / weight  # Estimate ISF using weight

        glucose_correction_insulin = glucose_error / ISF

        # Calculate insulin dosage using PID
        insulin_dosage = (pid_gains[i][0] * glucose_error +
                          pid_gains[i][1] * glucose_error +
                          pid_gains[i][2] * glucose_change) * 0.1
        insulin_dosage += glucose_correction_insulin

        # Apply a safe correction factor
        max_insulin = min(10 + (glucose_prediction - 110) / 10, 20)
        insulin_dosage = max(0, min(insulin_dosage + glucose_correction_insulin, max_insulin))

        print(f"Sample {i+1} - Predicted Glucose: {glucose_prediction:.2f} mg/dL")
        print(f"Recommended Insulin: {insulin_dosage:.2f} U (Safe Range)\n")

Main

In [35]:
if __name__ == "__main__":
    # Path to your saved model
    model_path = '/content/drive/MyDrive/GP PID/opt_pid_tuning_model.h5'

    # Load the model
    pid_model = load_model(model_path)

    # Test dataframe loading
    test_file = '/content/drive/MyDrive/GP PID/dataset/559-ws-testing.xml'
    test_df = parse_xml(test_file)

    # Preprocess time features
    test_df = preprocess_time_features(test_df)

    # Perform inference
    predict_insulin_dosage(pid_model, test_df)

Streaming output truncated to the last 5000 lines.
Recommended Insulin: 0.00 U (Safe Range)

Sample 819 - Predicted Glucose: 102.00 mg/dL
Recommended Insulin: 0.00 U (Safe Range)

Sample 820 - Predicted Glucose: 108.00 mg/dL
Recommended Insulin: 0.00 U (Safe Range)

Sample 821 - Predicted Glucose: 109.00 mg/dL
Recommended Insulin: 0.00 U (Safe Range)

Sample 822 - Predicted Glucose: 113.00 mg/dL
Recommended Insulin: 0.20 U (Safe Range)

Sample 823 - Predicted Glucose: 120.00 mg/dL
Recommended Insulin: 0.66 U (Safe Range)

Sample 824 - Predicted Glucose: 116.00 mg/dL
Recommended Insulin: 0.39 U (Safe Range)

Sample 825 - Predicted Glucose: 119.00 mg/dL
Recommended Insulin: 0.59 U (Safe Range)

Sample 826 - Predicted Glucose: 125.00 mg/dL
Recommended Insulin: 1.00 U (Safe Range)

Sample 827 - Predicted Glucose: 123.00 mg/dL
Recommended Insulin: 0.86 U (Safe Range)

Sample 828 - Predicted Glucose: 124.00 mg/dL
Recommended Insulin: 0.93 U (Safe Range)

Sample 829 - Predicted Glucose: 135.0